In [3]:
from langchain_voyageai import VoyageAIEmbeddings
import os
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
model = VoyageAIEmbeddings(batch_size=6, model='voyage-3', voyage_api_key=os.getenv('VOYAGEAI_API_KEY'))

In [6]:
from pymilvus import (connections,
                      Collection,
                      FieldSchema,
                      CollectionSchema,
                      DataType,
                      utility)

connections.connect(
    alias="default", 
    host="localhost",
    port="19530"
)

connections.disconnect(alias="default")

In [7]:
connections.disconnect(alias="default")

In [11]:

def create_collection(collection_name: str, data: list, dim: int):
    connections.connect("default", host="localhost", port="19530")
    if utility.has_collection(collection_name):
        utility.drop_collection(collection_name)

    fields = [
        FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=dim),
        FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=65535)
    ]

    schema = CollectionSchema(fields, description="Chunk Embeddings Collection")
    collection = Collection(name=collection_name, schema=schema)

    index_params = {
        "metric_type": "COSINE",
        "index_type": "IVF_FLAT",
        "params": {"nlist": 1024}
    }

    collection.create_index(field_name="embedding", index_params=index_params)
    collection.insert(data)
    collection.flush()
    collection.load()

    return collection

In [12]:
voyageai_collection = create_collection(collection_name="voyageai_embeddings", data=[embeddings_list, texts_list], dim=1024)

In [13]:
from pymilvus import MilvusClient

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

In [18]:
res = client.search(
    collection_name='voyageai_embeddings',
    data=[frontend_rfp_old_vectorized],
    anns_field='embedding',
    search_params={'metric_type': 'COSINE', "params": {"nprobe": 16}},
    limit=10,
    output_fields=['text']
)

hit_dict = {}
for hits in res:
    for hit in hits:
        print(hit['id'], hit['distance'], hit['entity']['text'][:18])
        hit_dict[hit['id']] = [hit['distance'], hit['entity']['text'][:18]]

455598750312170385 1.0 Key expert 2 - Con
455598750312170384 0.9105420112609863 Key expert 2 - Con
455598750312170390 0.8708397746086121 Key expert 3 - Con
455598750312170389 0.6366385817527771 FRONTEND DEVELOPER
455598750312170388 0.5711808204650879 FRONTEND DEVELOPER
455598750312170387 0.5515117645263672 BACKEND DEVELOPER 
455598750312170386 0.5263214707374573 BACKEND DEVELOPER

